## Student: Rodolfo Lerma

# Object Detection     

## Machine Learning 530   

### Steve Elston

## Introduction   

This lab will give you some hands on experience training a feature pyramid network object detector. The model you will work with is based on the RetinaNet model for multi-scale object detection.  

Before continuing, you will need to establish a Google Colab account. If you do not have an account you can sign up on [this page](https://colab.research.google.com/signup). A free account will be adequate for this lab. 

------------------
> **Note:** If you do not wish to run the notebook in Colab, you can download the example notebook and run it in another environment. Depending on the environment, doing so may require minor changes in the notebooks. 
------------------


## Starting and running the notebook

You will now start and execute a Jupyter notebook containing a Keras feature pyramid network object detector. Go to [this page](https://keras.io/examples/vision/retinanet/), and click **View in Colab**. Once the notebook launches in Colab read the provided commentary and examine the code for each cell. Then, execute the code all the cells in order. The model will be trained for 100 epochs, with each epoch requiring considerable computing time. 


> **Note:** For this assignment, you are required to submit this notebook with the exercises complete along with the executed notebook from Colab. To submit your executed notebook you must use File -> Download -> Download .ipynb. You can then upload your notebook into Canvas.  

> **Exercise 6-1:** Examine the code for the `AnchorBox` class and answer the following questions:     
> 1. What are the strides for the convolutional down-sampling pyramid?    
> 2. At each of these strides, what is the square root of the area of the anchor boxes?     
> 3. Why must the number of strides and areas specified be equal in terms of the feature pyramid?    
> 4. What are the aspect ratios of the bounding boxes? 
> 5. How many total anchor boxes are used in the feature pyramid network? Explain your calculation. **Hint**, the result should be a fairly small number, don't over-think this problem! 

> **Answers:**  
> 1. The strides are: [8, 16, 32, 64, 128]   
> 2. The areas are: [$32^{2}$,$64^{2}$,$128^{2}$,$256^{2}$,$512^{2}$] pixeles. 
> 3. Because that is going to determine the number of levels of the feature pyramid, therefore both have to match in number.
> 4. The aspect ratios for the bounding boxes are [0.5, 1.0, 2.0] or in other words [1:2, 1:1, 2:1]
> 5. 15 anchor boxes at each location on a given feature map. (3 ratios and 5 areas as noted by the "Feature Pyramid Networks for Object Detection" paper in section 4.1).  

> **Exercise 6-2:** Examine the code for the `FeaturePyramid` class. The `__init__` method creates the backbone CNN. Looking at the `get_backbone` function shows that it instantiates a ResNet50 model and returning feature maps at several depths from the model.  The `__init__` method then defines the convolution operators and an up-sample operation for the FPN. Keeping in mind the behavior of the [keras.layers.Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/) and [keras.layers.UpSampling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/UpSampling2D) functions, answer the following questions:   
> 1. How many feature maps from the ResNet50 backbone are used to construct the FPN? 
> 2. Do the tensors comprising the upward pyramid ever need to be stored in memory and why? 
> 3. How many scales levels with what number of channels comprise the downward (up-sampled) pyramid? 
> 4. Consider the steps to compute the `p3_output`, `p4_output`, and `p5_output`, tensors. Which of these tensors represents the smallest scale features and what is the reason this? Be specific in your references to the tensors comprising the upward and downward pyramid. Note that the numbering is from shallow to deep for the upward pyramid tensors.           
> 5. Which tensor in the downward pyramid represents the greatest semantic value features and what is the reason this? Be specific in your references to the tensors comprising the upward and downward pyramid.     

> **Answers:**   
> 1. From the original ResNet50 the model is using 3 feature maps: c3, c4 & c5. (In the research paper "Feature Pyramid Networks for Object Detection" the authors mentioned that they did not use c1 because its large memory footprint).
> 2. No, that is actually the main improvement of the use of Feature Pyramid Networks compared to the traditional Pyramid Representation for images. In this case the main advantage is that the convolution is by itself a pyramid structure where high dense information is extracted (or as the paper put it, the levels get semantically stronger at each section). Each convolution then talks to the next one and indirectly (by using the top-down pyramid model) with each other.
> 3. The entire `FeaturePyramid` provides 5 outputs (p3, p4, p5, p6, p7). The up-sample pyramid has 3 levels (p3, p4, p5) each one with 256 channels, by doing a 1 x 1 convolution and then a 3 x 3 convolution to reduce the alias-ing effect of up-sampling. 
> 4. The smallest scale feature is represented by `p5_output`, which is basically the last convolution taken form the backbound model (ResNet50), which therefore has the highest semantic value. In other words the model started to compressed the image and get higher semantic values starting from p3 (shallow) all the way to p5(deep). That is part of the reason we do not need to do upsample this feature map (because it is the deepest) 
> 5. As mentioned before the tensor with the greatest semantic value features is `p5_output` as this is the deepest level of the convolution (upward pyramid). Or in other words this tensor is the top of the pyramid that would be up-sampled going down by means of k-nearest neighbors. 

> **Exercise 6-3:** Examine the code for the loss functions used for this multi-task learning problem and provide short answers to the following questions:   
> 1. What kind of machine learning problem is solved for the bounding box parameter estimation task and what type of loss function is used and what are the advantages of this loss function for this problem in terms of noisy data and backpropagation for learning?        
> 2. What type of loss function is used for the object classification task and what is the advantage of this loss function necessary for this problem?    
> 3. From examination of just the last line of the `call` method how is the multi-task loss function computed. 

> **Answers:**  
> 1. The bounding box parameter estimation is a regression problem, and the model is using what is called `Smooth L1 Loss`.  It behaves as L1-loss (absolute_difference - 0.5) when the absolute value of the argument is high, and it behaves like L2-loss (0.5 * squared_difference) when the absolute value of the argument is close to zero. According to some research papers this loss is less sensitive to outliers and also prevents exploding gradients.
> 2. The object classification task as we can notice by the name is a classification problem and it is using `Focal Loss`. Based on the paper: "Focal Loss For Dense Object Detection" by Lin et al the main advantage is to improve the prediction accuracy for highly class imbalance problems. In other words the `Focal Loss` enables training highly accurate dense object detectors in the presence of vast numbers of easy background examples (areas with no detectable objects).
> 3. The multi-task loss (classification + bounding box) is computed by adding both losses: loss = clf_loss + box_loss, where:

clf_loss = tf.math.divide_no_nan(tf.reduce_sum(clf_loss, axis=-1), normalizer)
box_loss = tf.math.divide_no_nan(tf.reduce_sum(box_loss, axis=-1), normalizer)

> **Exercise 6-4:** Examine the two detection example images shown and answer these questions:    
> 1. For the bus image how many scales of objects are shown and what are they?   
> 2. For the objects detected in the bus image, which one has the highest confidence score and which one has the lowest confidence score. What are these scores? You will need to look carefully at the blue text.    
> 3. Why might the position of the objects in the image affect these confidence scores?  
> 4. For the image of the tennis player, how many scales of objects are shown and what are they?   

> **Answers:** 
> 1. In the image we can notice 3 objects detected, an each of this objects are in different scales:  the biggest scale is for the `bus` in the middle, follow by the `bus` in the left and third the `person` on the right. Technically speaking the 2 `bus`images seems to have  the same ratio but different scale, while the person is in a different ratio and scale all together (in comparison to the `bus` images)
> 2. The highest confidence score is given to the person with a value of 0.9, and the lowest is 0.76 which is for the bus on the left. These values represent the probability of the model that having selected that image as an object, that the object belong to specified category.        
> 3. This might depend on the training data set as well and in the preprocesing of the images (to capture more objects in different angles, sizes and positions). But any image that is front and center would have a better chance to be detected by the model and therefore potentially a better confidence score.
> 4. The image of the tennis player show 2 different kinds of bounding box in terms of ratio and each one is at a particular scale. But comparing in absolute values each box at different ratios, we might say that we have 2 scales (or 1 scale per box ratio): the `person` and the `tennis racket`.   

#### Copyright 2022, Stephen F Elston. All rights reserved. 